In [1]:
import json
import unqlite

In [2]:
DBPATH = '/tmp/test.db'

In [3]:
DATASET_PATH = '/home/d/bfly_data/pipelinetest/'

In [4]:
db = unqlite.UnQLite(DBPATH)

In [5]:
synapses = db.collection(DATASET_PATH+'_synapses')
neurons = db.collection(DATASET_PATH+'_neurons')

In [6]:
with open(DATASET_PATH+'synapse-connections.json', 'r') as f:
    input = json.load(f)

In [8]:
#
# SAMPLE DATA
#

# here we re-create the data
synapses.drop()
synapses.create()

neurons.drop()
neurons.create()

# wrangle data
for i,n1 in enumerate(input['neuron_1']):
    n2 = input['neuron_2'][i]
    x = input['synapse_center']['x'][i]
    y = input['synapse_center']['y'][i]
    z = input['synapse_center']['z'][i]

    # hack for neuron data
    neuron = {
        'id': n1,
        'x': x,
        'y': y,
        'z': z
    }
    if not neurons.filter(lambda n: (n['id']==neuron['id'])):
        neurons.store(neuron)
    neuron = {
        'id': n2,
        'x': x,
        'y': y,
        'z': z        
    }
    if not neurons.filter(lambda n: (n['id']==neuron['id'])):
        neurons.store(neuron)
    
    synapses.store({
        'presynaptic_neuron': n1,
        'postsynaptic_neuron': n2,
        'x': x,
        'y': y,
        'z': z
    })
    

# store to file
db.close()

In [ ]:
#### DENSE NEUROANATOMY ICD ENDPOINTS

In [76]:
#
# S2: get all synapse labels in region (BASED ON CENTERPOINTS)
#
x0,y0,z0 = [0,0,0]
x1,y1,z1 = [1024,1024,1024]

filtered_synapses = synapses.filter(lambda s: (s['x']>=x0 and s['x']<=x1) and 
                                              (s['y']>=y0 and s['y']<=y1) and 
                                              (s['z']>=z0 and s['z']<=z1))

synapse_ids = ['s'+str(s['__id']) for s in filtered_synapses]
print synapse_ids

['s0', 's1', 's2', 's3', 's7', 's8', 's9', 's13', 's14', 's15', 's17', 's18', 's20', 's21', 's22', 's29', 's31', 's38', 's39', 's40', 's41', 's42', 's44', 's45']


In [112]:
#
# S6: get all neuron labels in region (BASED ON CENTERPOINTS)
#
x0,y0,z0 = [0,0,0]
x1,y1,z1 = [1024,1024,1024]

filtered_neurons = neurons.filter(lambda n: (n['x']>=x0 and n['x']<=x1) and 
                                              (n['y']>=y0 and n['y']<=y1) and 
                                              (n['z']>=z0 and n['z']<=z1))

neuron_ids = ['n'+str(n['id']) for n in filtered_neurons]
print neuron_ids

['n9801', 'n9542', 'n7998', 'n389', 'n2814', 'n3826', 'n493', 'n557', 'n8526', 'n7855', 'n402', 'n4322', 'n4134', 'n7758', 'n533', 'n18', 'n5445', 'n13475', 'n471', 'n6487', 'n227', 'n10585', 'n404', 'n509', 'n265', 'n9224', 'n229', 'n6915', 'n85', 'n8454', 'n12164', 'n14', 'n3084', 'n8356', 'n15090']


In [113]:
#
# S9: get voxel list
#

### TODO, probably some np.where calls in a scalable matter


In [ ]:
### SPARSE ANATOMICAL DB ENDPOINTS

In [73]:
#
# S1: ID is a synapse
#
SYNAPSE_ID = 's0'

synapse = synapses.fetch(int(SYNAPSE_ID.strip('s')))
is_synapse = (synapse != None)
print str(is_synapse).lower()

true


In [69]:
#
# S3: synapse keypoint
#
SYNAPSE_ID = 's0'

# sanity check
synapse = synapses.fetch(int(SYNAPSE_ID.strip('s')))
is_synapse = (synapse != None)

if is_synapse:
    output = {
        'x': synapse['x'],
        'y': synapse['y'],
        'z': synapse['z']
    }
    
    print output


{'y': 341, 'x': 699, 'z': 31}


In [75]:
#
# S4: synapse parent
#
SYNAPSE_ID = 's0'


# sanity check
synapse = synapses.fetch(int(SYNAPSE_ID.strip('s')))
is_synapse = (synapse != None)

if is_synapse:
    output = {
        'synapse_id': 's'+str(synapse['__id']),
        'synapse_parent_pre': 'n'+str(synapse['presynaptic_neuron']),
        'synapse_parent_post': 'n'+str(synapse['postsynaptic_neuron']),
    }
    print output


{'synapse_parent_pre': 'n9801', 'synapse_id': 's0', 'synapse_parent_post': 'n9542'}


In [126]:
#
# S5: ID is a neuron
#
NEURON_ID = 'n9801'

neuron = neurons.filter(lambda n: n['id']==int(NEURON_ID.strip('n')))
is_neuron = not not neuron
print str(is_neuron).lower()

true


In [129]:
#
# S7: neuron keypoint
#
NEURON_ID = 'n9801'

neuron = neurons.filter(lambda n: n['id']==int(NEURON_ID.strip('n')))
is_neuron = not not neuron

if is_neuron:
    
    output = {
        'x': neuron[0]['x'],
        'y': neuron[0]['y'],
        'z': neuron[0]['z']
    }
    
    print output

{'y': 341, 'x': 699, 'z': 31}


In [131]:
#
# S8: neuron children (only presynaptic neurons?)
#
NEURON_ID = 'n9801'

neuron_synapses = synapses.filter(lambda s: s['presynaptic_neuron']==int(NEURON_ID.strip('n')))
valid_synapses = not not neuron_synapses

if valid_synapses:
    
    output = ['s'+str(s['__id']) for s in neuron_synapses]
    
    print output

['s0']
